# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 07**: Structured Streaming with Files 

**Date**: October 10th 2025

**Student Name**: José Alfredo Calvillo Gómez

**Professor**: Pablo Camarillo Ramirez

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr
import os

# Inicializar SparkSession
spark = SparkSession.builder \
    .appName("StructuredStreamingLab") \
    .getOrCreate()

# Directorio donde se guardan los logs
log_dir = "/ruta/a/tu/directorio/logs"  

# Definir el esquema para los datos de los logs (timestamp, nivel de log, y mensaje)
schema = "timestamp STRING, level STRING, message STRING"

# Leer datos del directorio de logs utilizando Structured Streaming
logs_df = spark.readStream \
    .schema(schema) \
    .csv(log_dir)

# Filtrar logs con errores críticos (por ejemplo, errores repetidos 500)
critical_errors_df = logs_df.filter(col("level") == "ERROR") \
    .filter(col("message").contains("500"))

# Contar los errores críticos por cada micro-lote
error_counts_df = critical_errors_df.groupBy("message").count()

# Sink: Mostrar el resultado en la consola
query = error_counts_df.writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

# Esperar a que el proceso de streaming termine (al menos 3 micro-lotes)
query.awaitTermination(30)  # Espera 30 segundos para que se procesen al menos 3 micro-lotes
